# Homework: Employee Analytics (NumPy + pandas + Matplotlib)

**Total: 100 points** • **Questions: 15**

This homework uses one dataset: **employee_data** (CSV). You will practice:

- NumPy arrays + vectorized operations
- pandas Series/DataFrame operations (filtering, cleaning, `groupby`, aggregates, pivots)
- Matplotlib plotting (pyplot + OO-style where appropriate)

**Dataset columns (expected):**

`Employee_ID, Age, Salary, Gender, Department, Experience, Education, Performance_Score, Working_Hours, City, Country, Years_in_Company, Previous_Company, Annual_Bonus, Join_Date, Name`

> You may assume `Employee_ID` is unique.


## Instructions (important)

- Use **pandas** for table operations, and **NumPy** for vectorized computations where asked.
- Do **not** hard-code answers; compute them from the data.
- Unless specified, treat missing values carefully (e.g., `NaN` in Hire Date/Bonus).
- Most questions ask you to create specific variables. **Name them exactly** as requested.
- Plots must have: title, axis labels, and readable tick labels.
- If you're using Google Colab, you need to upload the downloaded dataset to your Colab Files section.

### Grading

This notebook uses autograding:

- **Answer cells** are marked as `# YOUR CODE HERE` and will be graded.
- Remove `raise NotImplementedError()` once you start working on a solution.
- Only write your code where instructed. DO NOT ADD or REMOVE any cells. This may break the notebook.
- Do not edit the content of LOCKED cells.
- Besides the sanity check tests visible to you, we might use additional rigorous hidden tests that are only available after submission. Double-check your work for accuracy and do not rely on sanity checks.
- This notebook contains metadata and tracking.
- If anything breaks, save your work and download a fresh copy of the notebook from Canvas. You can copy your finished code and insert individually into appropriate cells of the new copy.


In [ ]:
# Setup
import hashlib
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

SALT = b"ml_io_hw2"


def dataframe_digest(df: pd.DataFrame) -> str:
    """
    Returns hexdigest of the hashed value of a given pands dataframe.
    """
    row_hashes = pd.util.hash_pandas_object(df, index=True)
    h = hashlib.sha256()
    h.update(row_hashes.values.tobytes())
    return h.hexdigest()


def hash_float(x, *, decimals=6, salt=SALT) -> str:
    """
    Hash a scalar float in a stable, tolerance-aware way.
    """
    x = float(np.round(x, decimals))
    h = hashlib.sha256()
    h.update(salt)
    h.update(np.float64(x).tobytes())
    return h.hexdigest()


def hash_value_counts(vc: pd.Series, *, salt=SALT) -> str:
    """
    Stable hash for value_counts() output.
    Order-insensitive and index-aware.
    """
    vc = vc.sort_index()
    row_hashes = pd.util.hash_pandas_object(vc, index=True).to_numpy(dtype=np.uint64)
    h = hashlib.sha256()
    h.update(salt)
    h.update(row_hashes.tobytes())
    return h.hexdigest()


pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

warnings.filterwarnings("ignore", category=FutureWarning)


## Load the dataset

Place your CSV in the same folder as this notebook and name it **`employee_data.csv`**.

If your file has a different name, change `DATA_PATH` accordingly.


In [ ]:
DATA_PATH = "employee_data.csv"

# Load
df = pd.read_csv(DATA_PATH)


# Basic cleanup: strip whitespace in column names and some string fields
df.columns = df.columns.str.strip()
for col in [
    "City",
    "Country",
    "Department",
    "Gender",
    "Education",
    "Name",
    "Previous_Company",
]:
    if col in df.columns:
        df[col] = df[col].astype("string").str.strip()

# Parse dates
if "Join_Date" in df.columns:
    df["Join_Date"] = pd.to_datetime(df["Join_Date"], errors="coerce")

assert (
    dataframe_digest(df)
    == "aeb852f872aa59e9be8e8afa192617ced6070fe7a45a30c02b6f7f158457a9bd"
), (
    "Your dataframe was not loaded correctly. Ensure your .csv file is not corrupted and try again."
)

df.head()

---

# Part A — Data understanding & cleaning (30 points)

### Q1 (4 pts) — Basic structure

Create the following variables:

- `n_rows`: number of rows
- `n_cols`: number of columns
- `colnames`: list of column names in order

**Hint:** `df.shape`, `df.columns`.


In [ ]:
# n_rows = ...
# n_cols = ...
# colnames = ...

# Be sure to remove raise NotImplementedError()
### BEGIN SOLUTION
n_rows, n_cols = df.shape
colnames = df.columns.tolist()
### END SOLUTION

In [ ]:
assert isinstance(n_rows, int) and n_rows > 0
assert isinstance(n_cols, int) and n_cols > 0
assert isinstance(colnames, list) and len(colnames) == n_cols

### BEGIN HIDDEN TESTS
assert n_rows == 100000
assert n_cols == 16
assert colnames == [
    "Employee_ID",
    "Age",
    "Salary",
    "Gender",
    "Department",
    "Experience",
    "Education",
    "Performance_Score",
    "Working_Hours",
    "City",
    "Country",
    "Years_in_Company",
    "Previous_Company",
    "Annual_Bonus",
    "Join_Date",
    "Name",
]
### END HIDDEN TESTS

### Q2 (6 pts) — Clean numeric columns

Ensure these columns are numeric (float or int), coercing errors to missing:

- `Salary`
- `Annual_Bonus`
- `Working_Hours`
- `Performance_Score`
- `Age`

Store the cleaned DataFrame as `df_clean` (do not overwrite `df`).

**Hint:** `pd.to_numeric(..., errors="coerce")`


In [ ]:
df_clean = ...

### BEGIN SOLUTION
df_clean = df.copy()
for c in ["Salary", "Annual_Bonus", "Working_Hours", "Performance_Score", "Age"]:
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")
### END SOLUTION

In [ ]:
assert isinstance(df_clean, pd.DataFrame), "df_clean must be a pandas DataFrame"

### BEGIN HIDDEN TESTS
required = ["Salary", "Annual_Bonus", "Working_Hours", "Performance_Score", "Age"]
for c in required:
    assert c in df_clean.columns
    assert pd.api.types.is_numeric_dtype(df_clean[c]), f"{c} must be numeric"
### END HIDDEN TESTS

In [ ]:
assert (
    dataframe_digest(df)
    == "aeb852f872aa59e9be8e8afa192617ced6070fe7a45a30c02b6f7f158457a9bd"
), (
    "The original dataframe is manpulated. Make sure you create new deep copies of dataframe and do not write back on it."
)

### Q3 (6 pts) — Missingness report

Create a DataFrame `missing_report` with columns:

- `missing_count`
- `missing_pct`

for every column in `df_clean`, sorted by `missing_pct` descending.

**Hint:** `isna().sum()`, divide by `len(df_clean)`.


In [ ]:
# You can first calculate and store the values of the columns in separate objects as intermediary step.
# missing_count = ...
# missing_pct = ...
# missing_report = ...

### BEGIN SOLUTION
missing_count = df_clean.isna().sum()
missing_pct = missing_count / len(df_clean) * 100
missing_report = pd.DataFrame(
    {"missing_count": missing_count, "missing_pct": missing_pct}
).sort_values("missing_pct", ascending=False)
### END SOLUTION

In [ ]:
assert isinstance(missing_report, pd.DataFrame)
assert (
    "missing_count" in missing_report.columns
    and "missing_pct" in missing_report.columns
)
assert missing_report.index.is_unique, (
    "the row index values of missing_report should be unique"
)

assert (
    hash_float(missing_report.sum()["missing_count"])
    == "ef9433bbeb1b4ee2103c0d6027351186102cb9ed8c18845575814fc3c9c454dc"
), "missing_count is not accurate."
assert (
    hash_float(missing_report.sum()["missing_pct"])
    == "4948daac02cee6d9928777e1a14764fc745fe1cc7ff431601132b8cd08564961"
), "missing_pct is not accurate"

### Q4 (7 pts) — Create derived features

Add the following columns to `df_clean`:

1. `Total_Comp` = `Salary` + `Annual_Bonus` (treat missing bonus as 0; missing salary should remain NaN)
2. `Overtime` = `Working_Hours` - 40 (but minimum 0; if Working_Hours is missing, Overtime should be NaN)
3. `Tenure_Bucket` based on `Years_in_Company`:
   - `"0-2"`, `"3-5"`, `"6-10"`, `"11+"`

Return the updated DataFrame as `df_feat`.

**Hints:**

- Bonus fill: `df["Annual_Bonus"].fillna(0)`
- For minimum 0: `np.maximum(...)`
- Buckets: `pd.cut(...)` or `np.select(...)`


In [ ]:
df_feat = df_clean.copy()

### BEGIN SOLUTION
# Total comp: treat missing bonus as 0, but keep Salary NaN if missing
df_feat["Total_Comp"] = df_feat["Salary"] + df_feat["Annual_Bonus"].fillna(0)

# Overtime: min 0; preserve NaN if Working_Hours is NaN
ot = df_feat["Working_Hours"] - 40
df_feat["Overtime"] = np.where(
    df_feat["Working_Hours"].isna(), np.nan, np.maximum(ot, 0)
)

# Tenure buckets (assumes Years_in_Company is numeric-ish; coerce to numeric defensively)
ten = pd.to_numeric(df_feat["Years_in_Company"], errors="coerce")
bins = [-np.inf, 2, 5, 10, np.inf]
labels = ["0-2", "3-5", "6-10", "11+"]
df_feat["Tenure_Bucket"] = pd.cut(ten, bins=bins, labels=labels)
### END SOLUTION


In [ ]:
assert isinstance(df_feat, pd.DataFrame)
assert all(c in df_feat.columns for c in ["Total_Comp", "Overtime", "Tenure_Bucket"]), (
    "All requested columns must be present in df_feat"
)
### BEGIN HIDDEN TESTS
assert str(df_feat["Tenure_Bucket"].dtype) == "category", (
    "Tenure_Bucket should be a categorical type"
)
### END HIDDEN TESTS

In [ ]:
assert (
    hash_float(df_feat["Total_Comp"].sum())
    == "66d803c4b4973e37762a2b1e4cd054aaec6b36288e18f1cbb83776b4604b5743"
), "Total_Comp values are not accurate."

assert (
    hash_float(df_feat["Overtime"].sum())
    == "ed09a59c80f1d2100797e9044eddda9697d021b70eb0fd51329f35179663e4e4"
), "Overtime values are not accurate."

In [ ]:
vc = (
    df_feat["Tenure_Bucket"]
    .value_counts()
    .reindex(["0-2", "3-5", "6-10", "11+"], fill_value=0)
)
assert (
    hash_value_counts(vc)
    == "c01867a9d50354bd6f66490c6f255825b3c056b7a4cde43d128e390710ea0f9f"
), "Tenure_Bucket distribution is not accurate."

### Q5 (7 pts) — Fix whitespace & categories

Create a DataFrame `df_cat` where:

- `Gender`, `Department`, `Country` are converted to pandas `category` dtype
- `City` is stripped of whitespace

Return `df_cat`.

**Hint:** `astype("category")`


In [ ]:
df_cat = df_feat.copy()

### BEGIN SOLUTION
for c in ["Gender", "Department", "Country"]:
    df_cat[c] = df_cat[c].astype("category")
df_cat["City"] = df_cat["City"].astype("string").str.strip()
### END SOLUTION

In [ ]:
for c in ["Gender", "Department", "Country"]:
    assert c in df_cat.columns
    assert str(df_cat[c].dtype) == "category"

---

# Part B — Analysis with aggregations & groupby (45 points)

### Q6 (6 pts) — Department salary summary

Create `dept_salary_summary` as a DataFrame indexed by `Department` with columns:

- `count` (non-missing salaries)
- `mean_salary`
- `median_salary`

Sorted by `mean_salary` descending.

**Hint:** `groupby("Department")["Salary"].agg(...)`


In [ ]:
# dept_salary_summary = ...
### BEGIN SOLUTION
dept_salary_summary = (
    df_cat.groupby("Department")["Salary"]
    .agg(count="count", mean_salary="mean", median_salary="median")
    .sort_values("mean_salary", ascending=False)
)
### END SOLUTION

In [ ]:
assert isinstance(dept_salary_summary, pd.DataFrame)
for c in ["count", "mean_salary", "median_salary"]:
    assert c in dept_salary_summary.columns, (
        f"column {c} not found in dept_salary_summary dataframe"
    )

### BEGIN HIDDEN TESTS
assert dept_salary_summary.shape == (5, 3), "The shape of the table is incorrect."
assert dept_salary_summary.index.to_list() == [
    "Marketing",
    "Finance",
    "HR",
    "Sales",
    "IT",
], "The indeces of the summary table are incorrect."
### END HIDDEN TESTS

In [ ]:
assert (
    hash_float(dept_salary_summary.loc["Marketing"].sum())
    == "2ff80081f15c05cadd08c9cedd95ed74111e173f4e43ddcb5383549b632f360a"
)
assert (
    hash_float(dept_salary_summary.loc["Finance"].sum())
    == "1b085536b0fe25b3f66022671c0146b13ad2e3114b0b4799ca071647dd37f7ca"
)
assert (
    hash_float(dept_salary_summary.loc["IT"].sum())
    == "45c0c84028dbd3f7f5d4137bbdb6a4bd7286a92e549795ec6027567473fa8f47"
)

assert (
    hash_float(dept_salary_summary.sum()["count"])
    == "5e59c5099a5ab591bd34d44c644824a0871b537a57134b9dafb725891a12b25f"
)
assert (
    hash_float(dept_salary_summary.sum()["mean_salary"])
    == "55346e7db096f48a134943698dedf67d3e0d2fb3a97f13e2693972f05e40f229"
)
assert (
    hash_float(dept_salary_summary.sum()["median_salary"])
    == "5ed6f2e6b9aa8bb00458bd2604cead292a9578028617c9c4538d26cfa202d1f0"
)

### Q7 (7 pts) — Gender pay gap by department

Compute a table `pay_gap_dept` with one row per department and columns:

- `female_mean_salary`
- `male_mean_salary`
- `gap_m_minus_f` = male_mean_salary - female_mean_salary

Departments missing either gender should still appear (gap can be NaN).

**Hint:** `pivot_table(index="Department", columns="Gender", values="Salary", aggfunc="mean")`


In [ ]:
# This template is just a suggestion but there are many ways to achieve this.
# pt = df_cat.pivot_table(...) # You can first create a pivot table ...
# pay_gap_dept = ...

### BEGIN SOLUTION
pt = df_cat.pivot_table(
    index="Department", columns="Gender", values="Salary", aggfunc="mean"
)
pay_gap_dept = pd.DataFrame(
    {
        "female_mean_salary": pt.get("Female"),
        "male_mean_salary": pt.get("Male"),
    }
)
pay_gap_dept["gap_m_minus_f"] = (
    pay_gap_dept["male_mean_salary"] - pay_gap_dept["female_mean_salary"]
)
pay_gap_dept = pay_gap_dept.sort_values("gap_m_minus_f", ascending=False)
### END SOLUTION

In [ ]:
assert isinstance(pay_gap_dept, pd.DataFrame)
for c in ["female_mean_salary", "male_mean_salary", "gap_m_minus_f"]:
    assert c in pay_gap_dept.columns, f"column {c} not found in pay_gap_dept dataframe"

### BEGIN HIDDEN TESTS
assert pay_gap_dept.shape == (5, 3), "The number of columns and rows is not as expected"
### END HIDDEN TESTS

In [ ]:
assert (
    hash_float(pay_gap_dept["female_mean_salary"].sum())
    == "f7a01ef2b10dae0100b76b50abfb3bb3ded638522d0d57bfe9b974ffe7fb09a5"
), "female_mean_salary is not accurate"
assert (
    hash_float(pay_gap_dept["male_mean_salary"].sum())
    == "5e4a1900fe96c402205b5a310650ffa5ed2a40f80dda494da4c6c2fe3d1cb124"
), "male_mean_salary is not accurate"
assert (
    hash_float(pay_gap_dept["gap_m_minus_f"].sum())
    == "183790c1c5c7f12e733348905d3512d20074326d7293800bd3dd1a629fcc3d77"
), "gap_m_minus_f is not accurate"

### BEGIN HIDDEN TESTS
row_sa = pay_gap_dept.loc["Sales"]
assert np.allclose(
    row_sa[["female_mean_salary", "male_mean_salary", "gap_m_minus_f"]]
    .astype(float)
    .to_numpy(),
    [75065.288, 75200.845, 135.557],
)

row_ma = pay_gap_dept.loc["Marketing"]
assert np.allclose(
    row_ma[["female_mean_salary", "male_mean_salary", "gap_m_minus_f"]]
    .astype(float)
    .to_numpy(),
    [75772.277, 75360.362, -411.915],
)
### END HIDDEN TESTS

### Q8 (7 pts) — Top earners per department [DEMO]

_This question is for exploration purposes only. Everyone will receive 7 points. Be sure to run the cell._

Create a DataFrame `top_earners` containing the **top 3** employees by `Total_Comp` within each `Department`.

Keep these columns (at least):
`Employee_ID, Name, Department, Salary, Annual_Bonus, Total_Comp`

Sort results by `Department` then `Total_Comp` descending.


In [ ]:
cols = ["Employee_ID", "Name", "Department", "Salary", "Annual_Bonus", "Total_Comp"]
tmp = df_cat[cols].sort_values(["Department", "Total_Comp"], ascending=[True, False])
top_earners = tmp.groupby("Department", group_keys=False).head(3).reset_index(drop=True)

# Be sure to remove raise NotImplementedError()
### BEGIN SOLUTION
### END SOLUTION

top_earners.head()

In [ ]:
assert isinstance(top_earners, pd.DataFrame)
assert top_earners.groupby("Department").size().max() <= 3
for c in ["Employee_ID", "Name", "Department", "Total_Comp"]:
    assert c in top_earners.columns

### Q9 (6 pts) — Correlation: Salary vs Performance [DEMO]

_This question is for exploration purposes only. Everyone will receive 6 points. Be sure to run the cell._

1. Compute overall Pearson correlation between `Salary` and `Performance_Score` as `corr_overall`.
2. Compute per-department correlations as a Series `corr_by_dept` indexed by department.

**Hints:**

- Overall: `df_cat[["Salary","Performance_Score"]].corr()` and extract the correlation from the resulting covariance matrix.
- By group: `groupby("Department").apply(lambda g: g["Salary"].corr(g["Performance_Score"]))` lambda functions are further exploration and advanced topic.


In [ ]:
corr_overall = df_cat[["Salary", "Performance_Score"]].corr().iloc[0, 1]
corr_by_dept = df_cat.groupby("Department").apply(
    lambda g: g["Salary"].corr(g["Performance_Score"])
)
print(f"Overall correlation: {corr_overall:.3f}")
print("\nCorelation by Department:\n")
print(corr_by_dept)

# Be sure to remove raise NotImplementedError()
### BEGIN SOLUTION
### END SOLUTION

In [ ]:
assert isinstance(corr_by_dept, pd.Series)
assert np.isclose(corr_overall, 0.075343)

### Q10 (6 pts) — Hiring trends: employees per year

Create a DataFrame `hires_per_year` with columns:

- `year`
- `n_hired`

derived from `Join_Date` (drop missing dates). Sorted by `year` ascending.

**Hint:** `df["Join_Date"].dt.year` and `value_counts().sort_index()`


In [ ]:
# You can use a few intermediary/helper objects
# years = ...
# value_counts = ...
# hires_per_year = ...

### BEGIN SOLUTION
years = df_cat["Join_Date"].dropna().dt.year
vc = years.value_counts().sort_index()
hires_per_year = vc.rename_axis("year").reset_index(name="n_hired")
### END SOLUTION

In [ ]:
assert isinstance(hires_per_year, pd.DataFrame)
assert set(["year", "n_hired"]).issubset(hires_per_year.columns)
### BEGIN HIDDEN TESTS
assert np.isclose(hires_per_year["year"].sum(), 28231), (
    "The values in column year are not accurate"
)
assert hires_per_year.shape == (14, 2), (
    "The number of columns and rows is not as expected"
)
assert np.isclose(hires_per_year["n_hired"].sum(), 98013), "n_hired is not accurate"
### END HIDDEN TESTS

### Q11 (7 pts) — Overtime analysis by department [DEMO]

_This question is for exploration purposes only. Everyone will receive the 7 points. Be sure to run the cell._

Using `Overtime` from Q4, create `overtime_dept` indexed by `Department` with:

- `avg_overtime`
- `pct_overtime_workers` = percent of employees in the department with overtime > 0 (exclude missing Working_Hours)

Sort by `avg_overtime` descending.

**Hints:**

- For percent: `(g["Overtime"] > 0).mean() * 100` but watch missing values
- Consider `dropna()` on Overtime for the percent denominator


In [ ]:
# In function definition what comes after : and -> signs (in parameters and after function)
# are type hints. They simply remind you what type goes in the function and what type is returned.
# Here g is pd.DataFrame type and the function returns a float type.
def pct_overtime(g: pd.DataFrame) -> float:
    s = g["Overtime"].dropna()
    if len(s) == 0:
        return np.nan
    return (s > 0).mean() * 100


# Notice the use of lambda (advanced topic).
overtime_dept = (
    df_cat.groupby("Department")
    .apply(
        lambda g: pd.Series(
            {
                "avg_overtime": g["Overtime"].mean(),
                "pct_overtime_workers": pct_overtime(g),
            }
        )
    )
    .sort_values("avg_overtime", ascending=False)
)

# Be sure to remove raise NotImplementedError()
### BEGIN SOLUTION
### END SOLUTION

overtime_dept

In [ ]:
assert isinstance(overtime_dept, pd.DataFrame)
for c in ["avg_overtime", "pct_overtime_workers"]:
    assert c in overtime_dept.columns

### Q12.1 (2 pts) — Education vs pay & performance

**A solution for this problem is mostly implemented. You will finish the solution and provide a written answer to the quesiton.**

We created a DataFrame `edu_summary` indexed by `Education` with:

- `n`
- `mean_salary`
- `mean_perf`

Sort this dataframe by `mean_salary` in descending order.


In [ ]:
edu_summary = df_cat.groupby("Education").agg(
    n=("Employee_ID", "count"),
    mean_salary=("Salary", "mean"),
    mean_perf=("Performance_Score", "mean"),
)
### BEGIN SOLUTION
edu_summary = edu_summary.sort_values("mean_salary", ascending=False)
### END SOLUTION

In [ ]:
assert isinstance(edu_summary, pd.DataFrame)
for c in ["n", "mean_salary", "mean_perf"]:
    assert c in edu_summary.columns, f"Column {c} not found in edu_summary"

### BEGIN HIDDEN TESTS
assert edu_summary["mean_salary"].is_monotonic_decreasing, (
    "mean_salary is not sorted descending (non-increasing)"
)
### END HIDDEN TESTS

### Q12.2 (4 pts) - What are your observations from this table? Why do you think we're seeing this trend in data?

_You may need to double-click this cell to edit._

### BEGIN SOLUTION

There is little difference between salary for educational groups. While such trend can very well exist in a real-world scenario, here it shows that our dataset is synthetic.
The authors of dataset could have used stratified sampling to inject structure into the synthetic data and better mimic the real-world.

### END SOLUTION


---

# Part C — Visualization (25 points)

### Q13 (8 pts) — Salary distribution by department (boxplot)

Create a **single figure** showing a **boxplot** of `Salary` by `Department`.

Requirements:

- Use either OO-style or pyplot style.
- Title: `"Salary Distribution by Department"`
- X-axis label: `"Department"`
- Y-axis label: `"Salary"`


In [ ]:
# This is just a suggested layout using OO-style approach.
# Feel free to use any other layout or pyplot approach.
# fig_q13, ax = plt.subplots(figsize=(8, 4))

### BEGIN SOLUTION
fig_q13, ax = plt.subplots(figsize=(8, 4))
# Drop missing salaries
plot_df = df_cat.dropna(subset=["Salary", "Department"])
# Prepare data grouped by department
depts = plot_df["Department"].astype(str).unique()
data = [
    plot_df.loc[plot_df["Department"].astype(str) == d, "Salary"].to_numpy()
    for d in depts
]
ax.boxplot(data, tick_labels=depts)
ax.set_title("Salary Distribution by Department")
ax.set_xlabel("Department")
ax.set_ylabel("Salary")
ax.tick_params(axis="x", rotation=30)
fig_q13.tight_layout()
### END SOLUTION

### Q14 (9 pts) — Trend plot: hires per year

Using `hires_per_year` from Q10, plot a line chart of `n_hired` vs `year`.

Requirements:

- Use **OO-style** (`fig, ax = plt.subplots()`)
- Markers visible on points
- Title: `"Employees Hired per Year"`
- X-axis label: `"Year"`
- Y-axis label: `"Number Hired"`

Store the Axes as `ax_q14`.


In [ ]:
### BEGIN SOLUTION
fig, ax_q14 = plt.subplots(figsize=(7, 4))
ax_q14.plot(hires_per_year["year"], hires_per_year["n_hired"], marker="o")
ax_q14.set_title("Employees Hired per Year")
ax_q14.set_xlabel("Year")
ax_q14.set_ylabel("Number Hired")
ax_q14.grid(True, alpha=0.3)
fig.tight_layout()
### END SOLUTION

In [ ]:
### BEGIN HIDDEN TESTS
from matplotlib.axes import Axes

assert isinstance(ax_q14, Axes), (
    "There must exist a matplotlib Axes object named ax_q14"
)
### END HIDDEN TESTS

### Q15 (8 pts) — Relationship plot: Salary vs Performance, colored by department

Create a scatter plot of `Salary` vs `Performance_Score`, with points colored by `Department`.

Requirements:

- Use Matplotlib (OO-style might be the better choice here)
- Include a legend with department names
- Title: `"Salary vs Performance (by Department)"`
- X-axis label: `"Salary"`
- Y-axis label: `"Performance Score"`

Store the figure as `fig_q15`.


In [ ]:
### BEGIN SOLUTION
fig_q15, ax = plt.subplots(figsize=(7, 5))
plot_df = df_cat.dropna(subset=["Salary", "Performance_Score", "Department"])

for dept, g in plot_df.groupby("Department"):
    ax.scatter(g["Salary"], g["Performance_Score"], label=str(dept), alpha=0.7)

ax.set_title("Salary vs Performance (by Department)")
ax.set_xlabel("Salary")
ax.set_ylabel("Performance Score")
ax.legend(title="Department", loc="best")
ax.grid(True, alpha=0.3)
fig_q15.tight_layout()
### END SOLUTION

---

## Submission checklist

- Restart kernel, run all cells top-to-bottom.
- Ensure **no errors**.
- Save notebook and submit the `.ipynb` file.
